# [Firedraka 安装](https://www.firedrakeproject.org/download.html)

需要计算机网络环境正常, 否则请参考 [无网络安装](#无网络安装)

<!--
jupyter nbconvert --to pdf 01_firedrake_install.ipynb

modified title:
https://github.com/jupyter/nbconvert/issues/249#issuecomment-636318699
-->

## Ubuntu

下载安装脚本 `firedrake-install` 然后运行即可

```bash
curl -O https://raw.githubusercontent.com/firedrakeproject/firedrake/master/scripts/firedrake-install
python3 firedrake-install
```

查看安装帮助
```bash
python3 firedrake-install -h
```

__注__: 有时安装会出现 pip 源不能访问的问题, 类似错误信息如下:

```shell
Starting new HTTPS connection (6): pypi.org:443
Could not fetch URL https://pypi.org/simple/pulp/: connection error: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pulp/ (Caused by NewConnectionError('<pip._vendor.urllib3.connection.HTTPSConnection object at 0x7f43dce52bc0>: Failed to establish a new connection: [Errno 101] Network is unreachable')) - skipping
Skipping link: not a file: https://pypi.org/simple/pulp/
Given no hashes to check 0 links for project 'pulp': discarding no candidates
ERROR: Could not find a version that satisfies the requirement PuLP (from versions: none)
ERROR: No matching distribution found for PuLP
```

可以通过设置 pip 的源解决, 如更改为 _中科大_ 的源:
```shell
mkdir -p $HOME/.pip && \
cat > $HOME/.pip/pip.conf <<EOF
[global]
index-url = https://pypi.mirrors.ustc.edu.cn/simple
[install]
trusted-host=pypi.mirrors.ustc.edu.cn
EOF
```

### Real Int32
```bash
python3 firedrake-install --slepc --disable-ssh --documentation-dependencies --remove-build-files
```

### Complex Int64
```bash
PETSC_CONFIGURE_OPTIONS='--download-scalapack --download-mumps' \
    python3 firedrake-install --petsc-int-type int64 --complex --slepc \
    --disable-ssh --documentation-dependencies --remove-build-files
```

### 测试

```shell
source firedrake/bin/activate
cd $VIRTUAL_ENV/src/firedrake
pytest tests/regression/ -k "poisson_strong or stokes_mini or dg_advection"
```

## Windows

安装 WSL (适用于 Linux 的 Windows 子系统, Windows Subsystem for Linux).
默认情况下，安装的系统为 Ubuntu. 

### WSL 安装

https://docs.microsoft.com/zh-cn/windows/wsl/install

### Firedrake 安装
按照 Ubuntu 安装方式

## Mac

先安装 Homebrew (https://brew.sh/), 然后使用 Homebrew 安装 python3, 之后类似于 Ubuntu 直接安装firedrake

## Linux Server

若服务器不能访问网络, 请参考下节: [无网络安装](#无网络安装).

### Spack I
Firedrake 团队提供了基于 Spack (HPC 上的包管理器) 安装的方式.

详见 : https://github.com/firedrakeproject/firedrake-spack

### Spack II

使用 spack 安装依赖包, 然后类似于 Ubuntu 方式安装 (需要禁用包管理器： `--no-package-manager`)

可参考如下脚本:

https://raw.githubusercontent.com/lrtfm/notes-for-firedrake/main/scripts/spack-firedrake.py

### Docker

可以使用 Firedrake 团队构建的镜像

https://hub.docker.com/u/firedrakeproject.

# 无网络安装

若需在无网络访问的工作站上安装 firedrake, 需要使用 spack 的镜像功能. 

假设本地可以 `git` 访问 github. 下面我们以安装软件在 `$HOME/opt` 目录为例.

注: 下面多行命令块中各行之间使用了 `&& \` 连接, 直接拷贝多行到终端输入回车即可.

Reference:

1. spack install:

    + https://spack.readthedocs.io/en/latest/getting_started.html#installation

2. spack mirror: 

    + https://spack.readthedocs.io/en/latest/mirrors.html#mirror-environment
    + https://spack.readthedocs.io/en/latest/mirrors.html#mirror-files

3. firedrake spack:

    + https://github.com/firedrakeproject/firedrake-spack
    + https://hackmd.io/@TzVnFeL0TMCb3FaAi9qYBA/ByaRskMQ5

## 本地

### 创建安装目录

```bash
mkdir $HOME/opt
```

### 下载安装 spack

```bash
cd $HOME/opt && \
git clone -c feature.manyFiles=true https://github.com/spack/spack.git && \
source $HOME/opt/spack/share/spack/setup-env.sh
```
        
注: 添加下面命令到文件 `$HOME/.bashrc` 中, 用于添加 spack 的 shell 支持, 使得每次开启终端都可以使用 spack.
```bash
source $HOME/opt/spack/share/spack/setup-env.sh
```

### 打包 spack 文件, 用于服务器安装
```bash
tar -czvf spack.tar.gz spack
```
        
### 下载 firedrake-spack 仓库

<!--
等待 `firedrake-spack` 修复
```bash
cd $HOME/opt && \
git clone https://github.com/firedrakeproject/firedrake-spack
git checkout tmp-fix-petsc-ptscotch
```
-->

```bash
cd $HOME/opt && \
git clone https://github.com/lrtfm/firedrake-spack.git && \
git checkout tmp-fix-petsc-ptscotch
```
                
### 打包 firedrake-spack 文件, 用于服务器安装
```bash
tar -czvf firedrake-spack.tar.gz firedrake-spack
```
        
### 添加该仓库到 `spack`
```bash
spack repo add firedrake-spack
```

### 检查 spack 安装情况
现在运行 `spack info py-firedrake` 查看 firedrake-spack 仓库是否添加成功
```shell
$ spack info py-firedrake
PythonPackage:   py-firedrake

Description:
    Firedrake is an automated system for the portable solution of partial
    differential equations using the finite element method (FEM)

Homepage: https://firedrakeproject.org

Preferred version:
    develop    [git] https://github.com/firedrakeproject/firedrake.git on branch master

Safe versions:
    develop    [git] https://github.com/firedrakeproject/firedrake.git on branch master

Deprecated versions:
    None

Variants:
    Name [Default]               When    Allowed values    Description
    =========================    ====    ==============    ===============================================

    64-bit-indices [off]         --      on, off           Install PETSc using 64bit indices
    build_system [python_pip]    --      python_pip        Build systems supported by the package
    complex [off]                --      on, off           Install Firedrake in complex mode
    minimal-petsc [off]          --      on, off           Build PETSc with minimal packages for Firedrake
    slepc [off]                  --      on, off           Install SLEPc and slepc4py

Build Dependencies:
    eigen            mpi            py-cython  py-h5py        py-mpi4py    py-pip        py-pyadjoint  py-scipy       py-sympy  py-vtk    slepc
    libspatialindex  petsc          py-fiat    py-islpy       py-numpy     py-pkgconfig  py-pyop2      py-setuptools  py-tsfc   py-wheel
    libsupermesh     py-cachetools  py-finat   py-matplotlib  py-petsc4py  py-progress   py-requests   py-slepc4py    py-ufl    python

Link Dependencies:
    eigen  libspatialindex  libsupermesh  mpi  petsc  python  slepc

Run Dependencies:
    eigen            petsc          py-finat       py-mpi4py    py-pip        py-pyop2         py-scipy       py-tsfc  slepc
    libspatialindex  py-cachetools  py-h5py        py-nbval     py-pkgconfig  py-pytest        py-setuptools  py-ufl
    libsupermesh     py-cython      py-islpy       py-numpy     py-progress   py-pytest-xdist  py-slepc4py    py-vtk
    mpi              py-fiat        py-matplotlib  py-petsc4py  py-pyadjoint  py-requests      py-sympy       python

```

当前 `$HOME/opt` 目录文件如下:
```shell
$ ls -lha
total 214M
drwxrwxr-x  4 z2yang z2yang  112 Oct 30 15:17 .
drwxrwxr-x  3 z2yang z2yang   47 Oct 30 15:02 ..
drwxrwxr-x  5 z2yang z2yang  204 Oct 30 15:16 firedrake-spack
-rw-rw-r--  1 z2yang z2yang 211K Oct 30 15:17 firedrake-spack.tar.gz
drwxrwxr-x 10 z2yang z2yang 4.0K Oct 30 15:17 spack
-rw-rw-r--  1 z2yang z2yang 214M Oct 30 15:15 spack.tar.gz
```

## 工作站

<!--
```bash
cd /home/z2yang/z2yang/server2
export HOME=`pwd`
export TERM=xterm
export PS1=' (server2) \w $ '
mkdir opt
cd opt
cp ../../local/opt/firedrake-spack.tar.gz .
cp ../../local/opt/spack.tar.gz .
```
-->

### 创建安装目录

```bash
mkdir $HOME/opt
```

### 上传文件

使用 ftp 等工具上传本地的 `firedrake-spack.gz` 和   `spack.tar.gz` 到服务器的 `$HOME/opt` 目录.


### 解压安装 spack

```bash
cd $HOME/opt && \
tar -zxf spack.tar.gz && \
source $HOME/opt/spack/share/spack/setup-env.sh
```
        
__注1__: 添加下面命令到文件 `$HOME/.bashrc` 中, 用于添加 spack 的 shell 支持, 使得每次开启终端都可以使用 spack.
```bash
source $HOME/opt/spack/share/spack/setup-env.sh
```

__注2__: 某些工作站上 `/tmp` 目录内容没有执行权限, 需要更改 spack 的构建目录配置如下:
```bash
mkdir -p $HOME/.spack && \
cat > $HOME/.spack/config.yaml <<EOF
config:
  build_stage:
    - \$user_cache_path/stage
EOF
```
        
### 解压安装 firedrake-spack 仓库
```bash
cd $HOME/opt && \
tar -zxf firedrake-spack.tar.gz && \
spack repo add firedrake-spack
```

### 准备 firedrake 安装环境

1. 创建环境

    ```bash
    FIREDRAKE_ENV_NAME=firedrake-complex-int64 && \
    spack env create -d $FIREDRAKE_ENV_NAME && \
    spack env activate -p $FIREDRAKE_ENV_NAME && \
    spack -e $SPACK_ENV config add concretizer:unify:true
    ```

2. 添加软件包 (可根据需要添加或删减)

    ```bash
    spack add python py-firedrake@develop%gcc +64-bit-indices+complex ^mpich ^openblas \
        ^petsc+mumps+scalapack+int64+complex+libyaml+parmmg+mmg \
        ^hypre+complex+int64+superlu-dist && \
    spack add py-pygmsh py-meshio py-tqdm py-pyyaml
    ```
    
    __注__: 需要显式添加 python, 不然 `view` 中 `python` 是链接而不是拷贝, 会导致 `firedrake` 找不到 `spatialindex`: https://github.com/spack/spack/issues/32456

4. 运行 `concretize` (spack 计算软件依赖关系)

    ```bash
    spack concretize -f 2>&1 | tee $SPACK_ENV/spack-firedrake-concretize.log
    ```

5. 查看 `$SPACK_ENV` 目录, 有如下内容

    ```bash
    $ ls -la $SPACK_ENV
    total 620
    drwxrwxr-x 3 z2yang z2yang    118 Oct 30 16:01 .
    drwxrwxr-x 5 z2yang z2yang    147 Oct 30 15:33 ..
    drwxrwxr-x 4 z2yang z2yang     89 Oct 30 16:01 .spack-env
    -rw-rw-r-- 1 z2yang z2yang  54343 Oct 30 16:01 spack-firedrake-concretize.log
    -rw-rw-r-- 1 z2yang z2yang 572917 Oct 30 16:01 spack.lock
    -rw-rw-r-- 1 z2yang z2yang    457 Oct 30 16:01 spack.yaml
    ```

### 本地创建镜像文件上传服务器
通过 `spack.lock` 文件在本地创建 firedrake 环境, 构建镜像, 并上传服务器.

1. 下载 `spack.lock` 文件到本地.

2. (本地)运行如下命令创建镜像 (创建镜像需要 10min 左右)

    ```bash
    cd $HOME/opt && \
    spack env create -d firedrake-mirror-env spack.lock && \
    spack env activate -p ./firedrake-mirror-env && \
    time spack mirror create -a -d spack-firedrake-mirror 2>&1 | tee creat-mirror.logs
    ```
    
    结束后会有如下输出
    
    ```bash
    ==> Summary for mirror in file:///home/z2yang/z2yang/local/opt/spack-firedrake-mirror
    ==> Archive stats:
        0    already present
        244  added
        0    failed to fetch.

    real    10m56.048s
    user    1m1.559s
    sys     0m13.604s
    ```

    如果有失败的, 需要先删除缓存, 重新创建镜像:
    
    ```bash
    spack clean -ds && \
    time spack mirror create -a -d spack-firedrake-mirror 2>&1 | tee creat-mirror.logs
    ```

3. (本地)打包镜像

    ```bash
    tar -czvf spack-firedrake-mirror.tar.gz spack-firedrake-mirror
    ```

4. 上传镜像
    
    使用 ftp 工具上传 `spack-firedrake-mirror.tar.gz` 到服务器上 `$HOME/opt` 目录

5. (服务器)解压镜像

    ```bash
    tar -xzvf spack-firedrake-mirror.tar.gz
    ```

6. (服务器)添加镜像路径到 spack

    ```shell
    cat > $HOME/.spack/mirrors.yaml <<EOF
    mirrors:
      local_filesystem: file://$HOME/opt/spack-firedrake-mirror
    EOF
    ```
    
7. (工作站) 查看镜像是否添加成功, 运行 `spack mirror lsit` 应该有如下信息  

    ```bash
    $ spack mirror list
    local_filesystem    file://<your-home-path>/opt/spack-firedrake-mirror
    spack-public        https://mirror.spack.io
    ```

### 安装 Firedrake

1. 运行 develop 命令以避免一些错误

    ```bash
    spack develop py-firedrake@develop && \
    spack develop libsupermesh@develop && \
    spack develop petsc@develop && \
    spack develop chaco@petsc && \
    spack develop py-fiat@develop && \
    spack develop py-finat@develop && \
    spack develop py-islpy@develop && \
    spack develop py-petsc4py@develop && \
    spack develop py-pyadjoint@develop && \
    spack develop py-pyop2@develop && \
    spack develop py-coffee@develop && \
    spack develop py-loopy@develop && \
    spack develop py-cgen@develop && \
    spack develop py-codepy@develop && \
    spack develop py-genpy@develop && \
    spack develop py-tsfc@develop && \
    spack develop py-ufl@develop
    ```

    <!-- __注__: 好像并不需要了, 不过还是加上以免出错. -->

2. 安装

   在服务器端运行下面命令安装 (第一次安装需要等待一段时间 1-2hour, 中间可能会失败, goog luck!)

    ```shell
    spack concretize -f 2>&1 | tee $SPACK_ENV/spack-firedrake-develop.log && \
    time spack install --fail-fast --show-log-on-error \
        --log-file $SPACK_ENV/spack-firedrake-install.log
    ```
    
    最后几行输出如下:

    ```shell
    [+] /home/z2yang/z2yang/server2/opt/spack/opt/spack/linux-ubuntu22.04-cascadelake/gcc-11.3.0/py-firedrake-develop-il3tmyuhh37rnaww3u2yxhxcqawp3hh6
    ==> Updating view at /home/z2yang/z2yang/server2/opt/firedrake-complex-int64/.spack-env/view

    real    184m44.242s
    user    836m56.348s
    sys     97m42.901s
    ```

3. 取消激活环境

    ```shell
    despacktivate
    ```
    
    会有如下 Warning 忽略即可

    ```shell
    $ despacktivate
    ==> Warning: Skipping reversal of unreversable operation<class 'spack.util.environment.UnsetEnv'> PETSC_ARCH
    ==> Warning: Skipping reversal of unreversable operation<class 'spack.util.environment.UnsetEnv'> PETSC_ARCH
    ==> Warning: Skipping reversal of unreversable operation<class 'spack.util.environment.UnsetEnv'> PETSC_ARCH
    ==> Warning: Skipping reversal of unreversable operation<class 'spack.util.environment.UnsetEnv'> PETSC_ARCH
    ```

### 使用
1. 激活环境

    ```shell
    cd $HOME/opt && \
    spack env activate -p $FIREDRAKE_ENV_NAME
    ```

2. 测试

    ```shell
    cd $SPACK_ENV/py-firedrake && \
    pytest tests/regression/ -k "poisson_strong or stokes_mini or dg_advection"
    ```

# Colab 上尝试 Firedrake 

Colab 是 Colaboratory 的简称,  (可以看作是在线版 Jupyter, 浏览器中编写和执行 Python 代码)

[FEM on Colab](https://fem-on-colab.github.io/index.html) 支持在 Colab 上安装 [FEniCS, FEniCSx, Firedrake, NGSolve, gmsh](https://fem-on-colab.github.io/packages.html)



## 导入 Package

### Firedrake

大概 3 分钟

```python
try:
    import firedrake
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" \
        -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    import firedrake
```

### Gmsh
```python
try:
    import gmsh
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/gmsh-install.sh" \
        -O "/tmp/gmsh-install.sh" && bash "/tmp/gmsh-install.sh"
    import gmsh
```

## 示例

https://colab.research.google.com/drive/1gM3zMWTskH7XyDi1yJL76BPFnOJjSdYh?usp=sharing

# 问题求助

__官方文档__: 

+ https://www.firedrakeproject.org/documentation.html

__Github 仓库 (issues and discussions)__: 

+ https://github.com/firedrakeproject/firedrake

+ https://github.com/firedrakeproject/firedrake/issues

+ https://github.com/firedrakeproject/firedrake/discussions

__Slack 讨论组__:

+ https://firedrakeproject.herokuapp.com/

邮件列表: 

+ https://mailman.ic.ac.uk/mailman/listinfo/firedrake

# 其他有限元工具

+ [FEniCSx](https://fenicsproject.org/)

+ [NgSolve](https://ngsolve.org/)

+ [deal.II](https://github.com/dealii/dealii)

+ [libMesh](https://libmesh.github.io/)

+ [FreeFEM](https://freefem.org/)

+ [Dune](https://www.dune-project.org/modules/dune-fem/)